In [1]:
import openai
from openai import cli
import time
import shutil
import json

In [2]:

# Remember to remove your key from your code when you're done.
openai.api_key = "9b5a04e88ddc47498e134c1c992253c4"
# Your resource endpoint should look like the following:
# https://YOUR_RESOURCE_NAME.openai.azure.com/
openai.api_base =  "https://openai-cosmize-instance.openai.azure.com/" 
openai.api_type = 'azure'
# The API version may change in the future.
openai.api_version = '2023-05-15'

training_file_name = 'output_1_prepared.jsonl'
validation_file_name = 'validation.jsonl'

# Copy the validation dataset file from the training dataset file.
# Typically, your training data and validation data should be mutually exclusive.
# For the purposes of this example, we're using the same data.
print(f'Copying the training file to the validation file')
shutil.copy(training_file_name, validation_file_name)

def check_status(training_id, validation_id):
    train_status = openai.File.retrieve(training_id)["status"]
    valid_status = openai.File.retrieve(validation_id)["status"]
    print(f'Status (training_file | validation_file): {train_status} | {valid_status}')
    return (train_status, valid_status)

# Upload the training and validation dataset files to Azure OpenAI.
training_id = cli.FineTune._get_or_upload(training_file_name, True)
validation_id = cli.FineTune._get_or_upload(validation_file_name, True)

# Check on the upload status of the training and validation dataset files.
(train_status, valid_status) = check_status(training_id, validation_id)

# Poll and display the upload status once a second until both files have either
# succeeded or failed to upload.
while train_status not in ["succeeded", "failed"] or valid_status not in ["succeeded", "failed"]:
    time.sleep(1)
    (train_status, valid_status) = check_status(training_id, validation_id)

Copying the training file to the validation file
Found potentially duplicated files with name 'output_1_prepared.jsonl', purpose 'fine-tune' and size 26302431 bytes
file-d990d47266a14ad188e2f3ac79f4c92d
file-e04422c251e24f099c475495e172d5f8
Enter file ID to reuse an already uploaded file, or an empty string to upload this file anyway: 

Upload progress: 100%|████████████████████| 26.3M/26.3M [00:00<00:00, 41.7Git/s]


Uploaded file from output_1_prepared.jsonl: file-ff383eeaefba4c2ca6024b6396422b84
Found potentially duplicated files with name 'validation.jsonl', purpose 'fine-tune' and size 26302431 bytes
file-e521176156934105b899976ee1b5af3a
file-e62153bfc776426880c85297973aa9c7
Enter file ID to reuse an already uploaded file, or an empty string to upload this file anyway: 

Upload progress: 100%|████████████████████| 26.3M/26.3M [00:00<00:00, 22.1Git/s]


Uploaded file from validation.jsonl: file-3330c01820f04f92bf7450989e0c6b70
Status (training_file | validation_file): succeeded | notRunning
Status (training_file | validation_file): succeeded | notRunning
Status (training_file | validation_file): succeeded | running
Status (training_file | validation_file): succeeded | succeeded


In [10]:
# This example defines a fine-tune job that creates a customized model based on davinci, 
# with just a single pass through the training data. The job also provides classification-
# specific metrics, using our validation data, at the end of that epoch.
create_args = {
    "training_file": training_id,
    "validation_file": validation_id,
    "model": "davinci",
    "compute_classification_metrics": True,
    "classification_n_classes": 3
}
# Create the fine-tune job and retrieve the job ID
# and status from the response.
resp = openai.FineTune.create(**create_args)
job_id = resp["id"]
status = resp["status"]

# You can use the job ID to monitor the status of the fine-tune job.
# The fine-tune job may take some time to start and complete.
print(f'Fine-tuning model with job ID: {job_id}.')

InvalidRequestError: The specified base model does not support fine-tuning.

In [7]:
# Get the status of our fine-tune job.
status = openai.FineTune.retrieve(id=job_id)["status"]

# If the job isn't yet done, poll it every 2 seconds.
if status not in ["succeeded", "failed"]:
    print(f'Job not in terminal status: {status}. Waiting.')
    while status not in ["succeeded", "failed"]:
        time.sleep(2)
        status = openai.FineTune.retrieve(id=job_id)["status"]
        print(f'Status: {status}')
else:
    print(f'Fine-tune job {job_id} finished with status: {status}')

# Check if there are other fine-tune jobs in the subscription. 
# Your fine-tune job may be queued, so this is helpful information to have
# if your fine-tune job hasn't yet started.
print('Checking other fine-tune jobs in the subscription.')
result = openai.FineTune.list()
print(f'Found {len(result)} fine-tune jobs.')

NameError: name 'job_id' is not defined

In [ ]:
# Retrieve the name of the customized model from the fine-tune job.
result = openai.FineTune.retrieve(id=job_id)
if result["status"] == 'succeeded':
    model = result["fine_tuned_model"]

# Create the deployment for the customized model, using the standard scale type without specifying a scale
# capacity.
print(f'Creating a new deployment with model: {model}')
result = openai.Deployment.create(model=model, scale_settings={"scale_type":"standard", "capacity": None})
# Retrieve the deployment job ID from the results.
deployment_id = result["id"]